<a href="https://colab.research.google.com/github/hoonzi-s/hongong_MLDL/blob/main/05-2.%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시작하기 전에

정형 데이터를 다루는 가장 성능 좋은 알고리즘: 앙상블 학습

# 랜덤 포레스트
앙상블 학습을 적용할 때 가장 먼저 시도할 알고리즘. 안정적인 성능

랜덤하게 만들어 트리 숲을 만들고 각 결정 트리의 예측을 사용해 최종 예측 결정

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
wine = pd.read_csv('https://bit.ly/wine_csv_data')
input = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']
train_input, test_input, train_target, test_target = tts(input, target, test_size = 0.2, random_state = 42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9973541965122431
0.8905151032797809


In [6]:
# 결정 트리의 장점 중 하나 - 특성 중요도 계산
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [8]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플: OOB(OUT OF BAG) 샘플
# OOB 샘플을 검증 세트로 활용
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
rf.oob_score_

0.8934000384837406

# 엑스트라 트리

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, n_jobs = -1, return_train_score = True)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9974503966084433
0.8887848893166506


In [11]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

#그라디언트 부스팅

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, n_jobs = -1, return_train_score = 42)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.8881086892152563
0.8720430147331015


In [13]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.11949946, 0.74871836, 0.13178218])

In [14]:
# 학습률, 트리 개수 증가
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, n_jobs = -1, return_train_score = True)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9464595437171814
0.8780082549788999


In [15]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

#히스토그램 기반 그라디언트 부스팅
그라디언트 부스팅의 속도와 성능 개선

In [16]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9321723946453317
0.8801241948619236


In [18]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result

{'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
         0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
        [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
         0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
        [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
         0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]]),
 'importances_mean': array([0.08876275, 0.23438522, 0.08027708]),
 'importances_std': array([0.00382333, 0.00401363, 0.00477012])}

In [19]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [20]:
hgb.score(test_input, test_target)

0.8723076923076923

In [23]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.8824322471423747
0.8726214185237284


In [24]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9338079582727165
0.8789710890649293
